In [1]:
"""
A Rankine vapor power cycle
"""

'\nA Rankine vapor power cycle\n'

In [2]:
import cantera as ct

In [3]:
# parameters
eta_pump = 0.6 # isentropic eff
eta_turbine = 0.8
p_max = 8.0e5

In [10]:
def pump(fluid, p_final, eta):
    """Adiabatically pump a fluid to pressure p_final, using
    a pump with isoentropic efficiency eta.
    """
    h0 = fluid.h
    s0 = fluid.s
    fluid.SP = s0, p_final
    h1s = fluid.h
    isentropic_work = h1s - h0
    actual_work = isentropic_work / eta
    h1 = h0 + actual_work
    fluid.HP = h1, p_final
    return actual_work

In [5]:
def expand(fluid, p_final, eta):
    """Adiabatically expand a fluid to pressure p_final, using
    a turbine with isentropic efficiency eta."""
    h0 = fluid.h
    s0 = fluid.s
    fluid.SP =s0, p_final
    h1s = fluid.h
    isentropic_work = h0 - h1s
    actual_work = isentropic_work * eta
    h1 = h0 - actual_work
    fluid.HP = h1, p_final
    return actual_work

In [6]:
def printState(n, fluid):
    print('\n***************** State {0} ******************'.format(n))
    print(fluid.report())

In [7]:
# create an object representing water
w = ct.Water()

In [8]:
# start with saturated liquid water at 300 K
w.TX = 300.0, 0.0
h1 = w.h
p1 = w.P
printState(1, w)


***************** State 1 ******************

  water:

       temperature             300  K
          pressure         3528.21  Pa
           density         996.589  kg/m^3
  mean mol. weight          18.016  amu
    vapor fraction               0

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy    -1.58582e+07       -2.857e+08     J
   internal energy    -1.58582e+07       -2.857e+08     J
           entropy          3913.2         7.05e+04     J/K
    Gibbs function    -1.70322e+07       -3.069e+08     J
 heat capacity c_p         1871.48        1.464e+09     J/K
 heat capacity c_v         1405.07        2.531e+04     J/K



In [11]:
# pump it adiabatically to p_max
pump_work = pump(w, p_max, eta_pump)
h2 = w.h
printState(2, w)


***************** State 2 ******************

  water:

       temperature         1447.24  K
          pressure          800000  Pa
           density         1.19796  kg/m^3
  mean mol. weight          18.016  amu
    vapor fraction               1

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy     -1.0892e+07       -1.962e+08     J
   internal energy    -1.15598e+07       -2.083e+08     J
           entropy         12859.5        2.317e+05     J/K
    Gibbs function    -2.95028e+07       -5.315e+08     J
 heat capacity c_p         2581.38        4.651e+04     J/K
 heat capacity c_v         2117.27        3.814e+04     J/K



In [12]:
# heat it at constant pressure until it reaches the saturated vapor state
# at this pressure
w.PX = p_max, 1.0
h3 = w.h
heat_added = h3 - h2
printState(3, w)


***************** State 3 ******************

  water:

       temperature         443.624  K
          pressure          800000  Pa
           density         4.15875  kg/m^3
  mean mol. weight          18.016  amu
    vapor fraction               1

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy    -1.32016e+07       -2.378e+08     J
   internal energy    -1.33939e+07       -2.413e+08     J
           entropy         10182.9        1.835e+05     J/K
    Gibbs function     -1.7719e+07       -3.192e+08     J
 heat capacity c_p     4.61687e+07        8.317e+08     J/K
 heat capacity c_v         1764.53        3.179e+04     J/K



In [13]:
# expand back to p1
turbine_work = expand(w, p1, eta_turbine)
printState(4, w)


***************** State 4 ******************

  water:

       temperature             300  K
          pressure         3528.21  Pa
           density       0.0305583  kg/m^3
  mean mol. weight          18.016  amu
    vapor fraction        0.835158

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy    -1.38221e+07        -2.49e+08     J
   internal energy    -1.39376e+07       -2.511e+08     J
           entropy         10700.1        1.928e+05     J/K
    Gibbs function    -1.70322e+07       -3.069e+08     J
 heat capacity c_p    inf              inf              J/K
 heat capacity c_v          108455        1.954e+06     J/K



In [14]:
turbine_work

620571.0158596382

In [15]:
# efficiency
eff = (turbine_work - pump_work)/heat_added
print('efficiency = ', eff)

efficiency =  0.8259978144917454


In [ ]:
# https://cantera.org/examples/python/thermo/rankine.py.html